# Generating `libveo.jl` using Clang.Generators

* [Generator Tutorial · Clang.jl](https://juliainterop.github.io/Clang.jl/stable/generator/)
  - 参考:
    - [VectorEngine.jl/clang.jl at master · sx-aurora-dev/VectorEngine.jl · GitHub](https://github.com/sx-aurora-dev/VectorEngine.jl/blob/master/deps/clang.jl)
    - [VectorEngine.jl/api.jl at 6bd99fc033962b5eb085e49fda8046c73a269513 · sx-aurora-dev/VectorEngine.jl](https://github.com/sx-aurora-dev/VectorEngine.jl/blob/6bd99fc033962b5eb085e49fda8046c73a269513/src/veda/api.jl)

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Linux (x86_64-redhat-linux)
  CPU: Intel(R) Xeon(R) Gold 6226 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cascadelake)
Environment:
  JULIA_DEPOT_PATH = /home/manabu/.julia-1.7.3


In [2]:
ENV["JULIA_DEPOT_PATH"]

"/home/manabu/.julia-1.7.3"

---

In [8]:
#import Pkg; Pkg.add("Clang")
using Clang.Generators

In [5]:
using Pkg
Pkg.status("Clang")

      Status `~/.julia-1.7.3/environments/v1.7/Project.toml`
  [40e3b903] Clang v0.15.8


* `ve_offload/libveo.toml` を用意する
  - 空の`[general]`のみ

In [15]:
options = load_options(joinpath(@__DIR__, "ve_offload/libveo.toml"))

Dict{String, Any} with 1 entry:
  "general" => Dict{String, Any}()

In [18]:
options["general"]["output_file_path"] = joinpath(@__DIR__, "ve_offload/libveo.jl");
options["general"]["library_name"] = "libveo";

In [19]:
ctx = create_context("/opt/nec/ve/veos/include/ve_offload.h", get_default_args(), options)

┌ Info: Found dependent header: /opt/nec/ve/veos/include/veo_hmem.h
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/context.jl:66
┌ Info: Parsing headers...
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/context.jl:88


Context(...)

In [20]:
build!(ctx)

┌ Info: Processing header: /opt/nec/ve/veos/include/ve_offload.h
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/passes.jl:33
┌ Info: Building the DAG...
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/context.jl:162
┌ Info: Emit Julia expressions...
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/context.jl:163
┌ Info: [ProloguePrinter]: print to /home/manabu/SX-Aurora_TSUBASA/Julia/doc/ve_offload/libveo.jl
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/passes.jl:1022
┌ Info: [GeneralPrinter]: print to /home/manabu/SX-Aurora_TSUBASA/Julia/doc/ve_offload/libveo.jl
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/passes.jl:948
┌ Info: [EpiloguePrinter]: print to /home/manabu/SX-Aurora_TSUBASA/Julia/doc/ve_offload/libveo.jl
└ @ Clang.Generators /home/manabu/.julia-1.7.3/packages/Clang/fCHPL/src/generator/passes.jl:1086
┌

Context(...)

## using `libveo.jl`

In [23]:
#import Pkg; Pkg.add("CEnum")
using CEnum

In [6]:
using Pkg
Pkg.status("CEnum")

      Status `~/.julia-1.7.3/environments/v1.7/Project.toml`
  [fa961155] CEnum v0.4.2


In [7]:
using Libdl
const libveo = Libdl.find_library("libveo.so.1", ["/opt/nec/ve/veos/lib64"])

"/opt/nec/ve/veos/lib64/libveo.so.1"

In [8]:
include("ve_offload/libveo.jl");

In [9]:
p = veo_proc_create(0)

Ptr{veo_proc_handle} @0x000000000205d3d0

* `libveo` を使うNLCPyとの互換性に配慮して、この`libveo.jl`は使わない･･･
  - `veo = pyimport("nlcpy.veo")`で同様の操作が可能

---

In [10]:
ctx = veo_context_open(p)

Ptr{veo_thr_ctxt} @0x0000000002099390

In [11]:
res = veo_load_library(p, "./_test.so")

[VE] ERROR: loadlib_handler() dlerror: ./_test.so: cannot open shared object file: No such file or directory


0x0000000000000000

In [ ]:
fun = veo_get_sym(p, res, "do_sleep")

In [12]:
args = veo_args_alloc()

Ptr{veo_args} @0x0000000002337af0

In [13]:
veo_args_set_i64(args, 0, 7)

0

* Passing Pointers for Modifying Inputs
  - [Calling C and Fortran Code · The Julia Language](http://gensoft.pasteur.fr/docs/julia/1.4.1/manual/calling-c-and-fortran-code.html#Passing-Pointers-for-Modifying-Inputs-1)

In [14]:
ret = Ref{UInt64}(0)

Base.RefValue{UInt64}(0x0000000000000000)

In [2]:
req = veo_call_async(ctx, fun, args)
veo_call_wait_result(ctx, req, ret)

* Juliaは `stdout` を吸収してしまう･･･